In [29]:
import importlib
from pathlib import Path

import instructor
import openai
import pandas as pd

from cuery import pprint, response, task, utils
from cuery.work import CountrySectors, DirceJobs, DirceTasks, SpanishSectors

GDRIVE = Path("~/Library/CloudStorage/GoogleDrive-victoriano@graphext.com/Shared drives/Solutions").expanduser()
task.Task.registry

{'SpanishSectors': <cuery.task.Task at 0x11b4d7150>,
 'CountrySectors': <cuery.task.Task at 0x11d4fb590>,
 'DirceJobs': <cuery.task.Task at 0x11b5b0510>,
 'DirceTasks': <cuery.task.Task at 0x11b5b31d0>}

In [25]:
from pydantic import Field
from enum import Enum
from cuery.prompt import Prompt
from cuery.task import Task

from cuery.response import ResponseModel

class SimpleSectors(ResponseModel):
    sectors: list[str] = Field(
        description="A list of 5 NAIC industrial sectors with their AI automation potential",
        min_length=5,
        max_length=5,
    )

prompt = Prompt(messages=[
    {"role": "user", "content": """ Please identify sectors of industries/companies in {{country}} that have great potential to be disrupted
        by AI (using AI software automation). Use the North American Industry Classification System (NAICS) to identify
        the NAMES of sector and subsector (not IDs). Focus on sectors with the biggest potential for disruption, employing a large
        number of jobs, and having the biggest number of companies."""}],
    required=["country"]
)

CountrySectors = Task(
    name="CountrySectors", prompt=prompt, response=SimpleSectors
)

In [26]:
context = [{"country": "Unites States"}, {"country": "Spain"}]
sectors_simple = await CountrySectors(context, n_concurrent=20)
sectors_simple

Gathering responses: 100%|██████████| 2/2 [00:01<00:00,  1.51it/s]


[SimpleSectors(sectors=['Professional, Scientific, and Technical Services', 'Health Care and Social Assistance', 'Manufacturing', 'Retail Trade', 'Finance and Insurance']), SimpleSectors(sectors=['Professional, Scientific, and Technical Services', 'Manufacturing', 'Wholesale Trade', 'Retail Trade', 'Real Estate and Rental and Leasing'])]

In [27]:
sectors_simple.to_pandas()

,country,sectors
0,Unites States,"Professional, Scientific, and Technical Services"
1,Unites States,Health Care and Social Assistance
2,Unites States,Manufacturing
3,Unites States,Retail Trade
4,Unites States,Finance and Insurance
5,Spain,"Professional, Scientific, and Technical Services"
6,Spain,Manufacturing
7,Spain,Wholesale Trade
8,Spain,Retail Trade
9,Spain,Real Estate and Rental and Leasing


In [30]:
DATA_DIR = GDRIVE / "Research/future_of_work/inputs/ine_dirce_aggregated_by_activity.parquet"
data = pd.read_parquet(DATA_DIR)
data.columns = [c.replace(" ", "_") for c in data.columns]
data = data.rename(columns={"Division": "sector", "Actividad_principal": "subsector"})
display(data)

n_examples = 5
df = data.iloc[:5]

,sector,subsector,Estimated_Employees_2024,Estimated_Employees_pct,Median_YoY_Growth_pct,Growth_2020_2024_pct,Total_2024,Total_2023,Total_2022,Total_2021,...,Estrato_De_50_a_99_pct,Estrato_De_100_a_199_pct,Estrato_De_200_a_249_pct,Estrato_De_250_a_999_pct,Estrato_De_1000_a_4999_pct,Estrato_De_5000_o_más_asalariados_pct,Condicion_Sociedades_anónimas_abs,Condicion_Sociedades_de_responsabilidad_limitada_abs,Condicion_Sociedades_anónimas_pct,Condicion_Sociedades_de_responsabilidad_limitada_pct
0,"05 Extracción de antracita, hulla y lignito",051 Extracción de antracita y hulla,216.0,0.00,-12.0,-61.1,21,23,44,52,...,4.76,0.00,0.00,0.00,0.00,0.00,7,14,33.33,66.67
1,"05 Extracción de antracita, hulla y lignito",052 Extracción de lignito,2.0,0.00,0.0,-33.3,2,3,3,3,...,0.00,0.00,0.00,0.00,0.00,0.00,2,0,100.00,0.00
2,06 Extracción de crudo de petróleo y gas natural,061 Extracción de crudo de petróleo,16.0,0.00,16.7,100.0,12,6,4,5,...,0.00,0.00,0.00,0.00,0.00,0.00,9,3,75.00,25.00
3,06 Extracción de crudo de petróleo y gas natural,062 Extracción de gas natural,8.0,0.00,-50.0,0.0,2,2,0,1,...,0.00,0.00,0.00,0.00,0.00,0.00,0,2,0.00,100.00
4,07 Extracción de minerales metálicos,071 Extracción de minerales de hierro,84.0,0.00,-17.0,-52.6,9,11,13,16,...,0.00,0.00,0.00,0.00,0.00,0.00,3,6,33.33,66.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,94 Actividades asociativas,942 Actividades sindicales,0.0,0.00,0.0,0.0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN
248,94 Actividades asociativas,949 Otras actividades asociativas,49.0,0.00,25.0,100.0,2,3,2,1,...,0.00,0.00,0.00,0.00,0.00,0.00,0,2,0.00,100.00
249,"95 Reparación de ordenadores, efectos personal...",951 Reparación de ordenadores y equipos de com...,14968.0,0.13,-2.8,-12.6,2153,2097,2324,2387,...,0.51,0.19,0.09,0.14,0.09,0.00,31,2122,1.44,98.56
250,"95 Reparación de ordenadores, efectos personal...",952 Reparación de efectos personales y artícul...,14278.0,0.12,-1.0,-9.3,3328,3336,3592,3604,...,0.21,0.06,0.03,0.03,0.03,0.00,31,3297,0.93,99.07
